# **TOXIC COMMENT CLASSIFIER USING LSTM**

# **Shubham Fulzele(1001947006)**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import re
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords          
from nltk.stem import WordNetLemmatizer   
from nltk.corpus import stopwords       
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split



# Importing Dataset

In [ ]:
training = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
testing = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")

In [ ]:
training.info()

In [ ]:
testing.info()

# Check for any NULL values present in both the training and testing dataset.

In [ ]:
training.isnull().sum()


In [ ]:
testing.isnull().sum()

No null values were found, so we are good to go ahead

# **Splitting the dataset**

In [ ]:
X_train = training["comment_text"]

y_train = training[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

X_test = testing["comment_text"]


In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

**Data Visualization**

In [ ]:
feature = training.drop(['id', 'comment_text'], axis=1)     ### Removed unnecessary columns - id and comment_text
counts = []                                               ### A list that contains tuple which consists of class label and number of comments for that particular class 
categories = list(feature.columns.values)
for i in categories:
    counts.append((i, feature[i].sum()))
    
df_1 = pd.DataFrame(counts, columns=['Feature Labels', 'Total Comments'])   ### Dataframe made up of category and total number of comments
df_1.plot(x='Feature Labels', y='Total Comments', kind='bar',figsize=(8,8))
plt.title("Comments per category")
plt.ylabel('Total comments', fontsize=12)
plt.xlabel('Feature Labels', fontsize=12)


In [ ]:
import seaborn as sns
plt.figure(figsize=(10,8)) 
sns.heatmap(feature.corr(), annot =True, fmt='.0%')

**Data Cleaning**

In [ ]:
length = training['comment_text'].str.len()

print("Max length of a string is {}".format(length.max()))
print("Mean length of a string is {}".format(length.mean()))
print("Minimum length of a string is {}".format(length.min()))

In [ ]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
remove_n = lambda x: re.sub("\n", " ", x)
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)
training['comment_text'] = training['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
training['comment_text'][0]

In [ ]:
testing['comment_text'] = testing['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
testing['comment_text'][0]

In [ ]:
X_train = training["comment_text"]

y_train = training[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

X_test = testing["comment_text"]


**MODEL**

In [ ]:
def Token(X,y):
    tok = Tokenizer(num_words = 20000)
    tok.fit_on_texts(list(X))
    tok_train = tok.texts_to_sequences(X)
    tok_test = tok.texts_to_sequences(y)
    return tok_train, tok_test


# **Model_0**

In [ ]:
tokenized_train,tokenized_test= Token(X_train,X_test)

maxlen = 150
x_train = pad_sequences(tokenized_train, maxlen = maxlen)
x_test = pad_sequences(tokenized_test, maxlen = maxlen)

In [ ]:
def LSTM_Model_0(maxlen, max_feature):
    pretrained_weights = Input(shape = (maxlen,))
    embed_size = 128
    x = LSTM(120, return_sequences = True, name = "lstm_layer")
    x = Embedding(max_feature, embed_size)(pretrained_weights)
    x = GlobalMaxPool1D()(x)
    x = Dense(64, activation = "relu")(x)
    x = Dense(32, activation = "relu")(x)
    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = pretrained_weights, outputs = x)
    model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
    return model


In [ ]:
lstm_0=LSTM_Model_0(150, 20000)

batch_size = 128
epochs = 10
history_0 = lstm_0.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.3, shuffle=True)


In [ ]:
tf.keras.utils.plot_model(lstm_0)

In [ ]:
loss = history_0.history['loss']
val_loss = history_0.history['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

In [ ]:
loss = history_0.history['accuracy']
val_loss = history_0.history['val_accuracy']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Accuracy')
plt.plot(epochs, val_loss, label='Validation Accuracy')
plt.title('Accuracy and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show();

# **Model_1**

Using Drop Out Functions to avoid Overfitting of the model

In [ ]:
tokenized_train,tokenized_test= Token(X_train,X_test)

maxlen = 150
x_train = pad_sequences(tokenized_train, maxlen = maxlen)
x_test = pad_sequences(tokenized_test, maxlen = maxlen)

In [ ]:
def LSTM_Model_1(maxlen, max_feature):
    pretrained_weights = Input(shape = (maxlen,))
    embed_size = 128
    x = LSTM(80, return_sequences = True, name = "lstm_layer")
    x = Embedding(max_feature, embed_size)(pretrained_weights)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation = "sigmoid")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = pretrained_weights, outputs = x)
    model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
    return model


In [ ]:
lstm_1=LSTM_Model_1(150, 20000)

batch_size = 512
history_1 = lstm_1.fit(x_train, y_train, batch_size = batch_size, epochs = 10, validation_split = 0.3, shuffle=True)


In [ ]:
tf.keras.utils.plot_model(lstm_1)

In [ ]:
loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

In [ ]:
loss = history_1.history['accuracy']
val_loss = history_1.history['val_accuracy']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

# **MODEL_3**

In [ ]:
def LSTM_Model_3(maxlen, max_feature):
    pretrained_weights = Input(shape = (maxlen,))
    embed_size = 128
    x = LSTM(30, return_sequences = True, name = "lstm_layer")
    x = Embedding(max_feature, embed_size)(pretrained_weights)
    x = GlobalMaxPool1D()(x)
    x = Dense(64, activation = "relu")(x)
    x = Dense(32, activation = "sigmoid")(x)
    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = pretrained_weights, outputs = x)
    model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])
    return model


In [ ]:
lstm_3=LSTM_Model_3(150, 20000)

batch_size = 90
history = lstm_3.fit(x_train, y_train, batch_size = batch_size, epochs = 6, validation_split = 0.3, shuffle=True)


In [ ]:
loss = history.history['accuracy']
val_loss = history.history['val_accuracy']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show();

# Predicting output of test set

In [ ]:
y_pred = lstm_3.predict(x_test)

In [ ]:

submiss_df= pd.DataFrame(y_pred, columns= ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
submiss_df.sample(n=6)

In [ ]:

target_cols = np.array(['toxic','severe_toxic','obscene', 'threat','insult', 'identity_hate'])
final_predict_test = pd.concat([pd.DataFrame(y_pred, columns=target_cols)], 1)
t1 = testing['id']
final_predict_test = pd.concat([t1,final_predict_test],1)
final_predict_test.sample(n=4)


In [ ]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)